#  Introduction

In this tutorial, we will train logistic/softmax regression model using TensorFlow, to predict if the input is neutral face or smiling face.
About the dataset, I collected images from Google Images and cropped the mouth part is manually put them into two separate folders. 
This is developed in Python 2.7.10 and Tensorflow 1.2.1

In [44]:
import cv2
import glob,os
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
print "tensorflow version",tf.__version__
%matplotlib inline

tensorflow version 1.2.1


# Loading data and resize 

In [21]:
# reading images from the data folder
# label, smile = 1, neutral = 0
(img_width, img_height) = (45, 25)
def load_images_from_folder(folder):
    (images, lables, names, id) = ([], [], {}, 0)
    for (subdirs, dirs, files) in os.walk(folder):
        print subdirs, dirs
        for subdir in dirs:
            names[id] = subdir
            subjectpath = os.path.join(folder, subdir)
            for filename in os.listdir(subjectpath):
                path = subjectpath + '/' + filename
                lable = id
                img = cv2.imread(path, 0) # Reading each images in grayscale
                img = cv2.resize(img,(img_width, img_height)) # Resizing all the images
                images.append(img)
                lables.append(int(lable))
            id += 1
        
        return images, lables, names


In [41]:
X,Y,classes = load_images_from_folder("./data/")

(X,Y) = [np.array(lis) for lis in [X, Y]]
Y = pd.get_dummies(Y) #converting labels to one-hot, Used Pandas for it. 

./data/ ['neutral', 'smile']


# Dimension of our data

In [23]:
print "Height of the image:  " + str(X.shape[1])
print "Width of the image:   "  + str(X.shape[2])
print "Total number of data: "+ str(len(X))

Height of the image:  25
Width of the image:   45
Total number of data: 163


# Splitting the data using sklearn 

In [24]:
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.33, random_state=42)

# Size of data, Train set and Test set

In [25]:
print "lenght of train data: "+str(len(X_train))
print "lenght of test data:  "+str(len(X_test))

lenght of train data: 109
lenght of test data:  54


# Flatting the data into 1 dimension

While we implement Softmax regression, it need 1 dimension array as an Input, It's important to convert our 2-D array to 1-D array
<img src="flattened_image.jpg">

In [26]:
train_set_x_flatten = X_train.reshape(X_train.shape[0],-1)
test_set_x_flatten = X_test.reshape(X_test.shape[0],-1)
print "train_set_x_flatten: "+str(train_set_x_flatten.shape)
print "test_set_x_flatten : "+str(test_set_x_flatten.shape)

print train_set_x_flatten

train_set_x_flatten: (109, 1125)
test_set_x_flatten : (54, 1125)
[[172 177 177 ...,  92 104 102]
 [155 158 161 ..., 162 159 155]
 [173 160 137 ..., 143 136 125]
 ..., 
 [150 150 143 ..., 160 158 159]
 [134 142 148 ..., 154 147 153]
 [ 28  29  30 ...,  78  77  58]]


In [46]:
#Normalizing data, conveting all pixel value in range between 0-1
train_set_x = train_set_x_flatten/255.
test_set_x = test_set_x_flatten/255.
print "length of train set: " + str(len(train_set_x))
print "length of test set: " + str(len(test_set_x))
print "shape of train set: "+ str(train_set_x.shape)
print "shape of test set: "+ str(test_set_x.shape)


length of train set: 109
length of test set: 54
shape of train set: (109, 1125)
shape of test set: (54, 1125)


# Building the softmax regression using tensorflow

In [29]:
import tensorflow as tf

In [30]:
x = tf.placeholder(tf.float32, [None, 1125])
W = tf.Variable(tf.zeros([1125, 2]), dtype=tf.float32, name="w1")
b = tf.Variable(tf.zeros([2]),  dtype=tf.float32, name="bias")
y = tf.nn.softmax(tf.matmul(x, W) + b, name="first_operation")
y_ = tf.placeholder(tf.float32, [None, 2], name = "final_output")
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

In [49]:
learning_rate = 0.2
print "training with learning rate: " + str(learning_rate)
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)

saver = tf.train.Saver()
with tf.Session() as sess:
    
    sess.run(tf.global_variables_initializer())
    for step in range(15000):
        trainStep,loss = sess.run([train_step, cross_entropy], feed_dict={x: train_set_x, y_: y_train})
        if step%1000==0:
            print "losses after per 1000 iteration: ",loss

    save_path = saver.save(sess, "./model/emotion_model")

    print("Model saved in file: %s" % save_path)

    correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print("accuracy with learning rate: " ,str(learning_rate), sess.run(accuracy, feed_dict={x:test_set_x , y_: y_test}))

training with learning rate: 0.2
losses after per 1000 iteration:  0.693147
losses after per 1000 iteration:  0.00812404
losses after per 1000 iteration:  0.00456266
losses after per 1000 iteration:  0.00328482
losses after per 1000 iteration:  0.00259862
losses after per 1000 iteration:  0.00216482
losses after per 1000 iteration:  0.00186346
losses after per 1000 iteration:  0.00164081
losses after per 1000 iteration:  0.00146906
losses after per 1000 iteration:  0.00133228
losses after per 1000 iteration:  0.00122064
losses after per 1000 iteration:  0.00112772
losses after per 1000 iteration:  0.00104913
losses after per 1000 iteration:  0.000981756
losses after per 1000 iteration:  0.00092333
Model saved in file: ./model/emotion_model
('accuracy with learning rate: ', '0.2', 0.51851851)


# prediction 

In [40]:
# W = tf.Variable(tf.zeros([1125, 2]),dtype = tf.float32, name="w1")
# b = tf.Variable(tf.zeros([2]), dtype = tf.float32,name="bias")
saver = tf.train.Saver()
with tf.Session() as sess:
    saver.restore(sess, "./model/emotion_model")
    print("w1:", sess.run(W))
    print("bias:", sess.run(b))
    img = cv2.imread("/home/aquib/Desktop/smile_recognition/test/test_5n.jpg", 0) # Reading each images in grayscale 
                                                                          # test_2 is a neutral face
                                                                            # test_1 is smile face
    img = cv2.resize(img,(img_width, img_height)) # Resizing all the images
    image=[]
    image.append(img)
    image_test = np.array(image)
    test_image_flatten = image_test.reshape(image_test.shape[0],-1)
    test_image_normalized = test_image_flatten/255.
    print test_image_normalized.shape
    x_ = tf.cast(test_image_normalized, tf.float32)
    y = tf.nn.softmax(tf.matmul(x_, W) + b)
    print sess.run(y)
    indx = sess.run(tf.argmax(y,1))
    if indx==1:
        print "smile"
    elif indx==0:
        print "neutral"

INFO:tensorflow:Restoring parameters from ./model/emotion_model
('w1:', array([[ 0.2276748 , -0.22767468],
       [ 0.14447214, -0.14447215],
       [ 0.13912062, -0.13912085],
       ..., 
       [-0.09384812,  0.09384794],
       [ 0.00484194, -0.00484221],
       [ 0.0309796 , -0.0309797 ]], dtype=float32))
('bias:', array([ 0.04066655, -0.04066731], dtype=float32))
(1, 1125)
[[  9.99999762e-01   2.05811475e-07]]
neutral
